In [1]:
;pwd

/Users/haocheng/Github/JWAS.jl/docs/notebooks


In [2]:
include("/Users/haocheng/Github/JWAS.jl/src/JWAS.jl")

JWAS

In [3]:
mypath    ="/Users/haocheng/Box Sync/5.Data/rainbowtrout/complete_genomic_data/"
phenofile =mypath*"pheno.common.csv"
genofile  =mypath*"geno.common.csv"
pedfile   =mypath*"ped.txt"

cd(mypath)

using DataFrames,CSV,JWAS
phenotype = CSV.read(phenofile,delim = ',',header=true);
phenotype

#priors
residual_variance             = 20.0
genetic_variance              = 20.0
genetic_variance_by_marker    = genetic_variance*0.5
genetic_variance_by_polygenic = genetic_variance*0.5
variance_family_data          = 20.0
familiy_date_variance         = 20.0

#build model
pedigree = get_pedigree(pedfile);
model = build_model("days = intercept + family_date + animal",residual_variance)
set_random(model,"family_date",variance_family_data)
set_random(model,"animal",pedigree,genetic_variance_by_polygenic)
add_genotypes(model,genofile,genetic_variance_by_marker,header=false,separator=',')

output=runMCMC(model,phenotype,chain_length=500,burnin=100,printout_frequency=100,methods="BayesB",Pi=0.9,output_samples_frequency=1);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


42045 markers on 992 individuals were added.
The prior for marker effects variance was calculated from genetic varaince and π.
Marker effects variance is 
0.007796.


MCMC Information:

methods                                      BayesB
chain_length                                    500
burnin                                          100
starting_value                                false
printout_frequency                              100
output_samples_frequency                          1
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                                               0.9
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marke

running MCMC for BayesB ... 40%|██████████              |  ETA: 0:00:38

...

running MCMC for BayesB ... 80%|███████████████████     |  ETA: 0:00:13


Posterior means at iteration: 400
Residual variance: 27.634945
Polygenic effects covariance matrix 
[5.375]


running MCMC for BayesB ...100%|████████████████████████|  ETA: 0:00:00


Posterior means at iteration: 500
Residual variance: 27.630992
Polygenic effects covariance matrix 
[5.44]


running MCMC for BayesB ...100%|████████████████████████| Time: 0:01:07


In [4]:
using JWAS:misc
file=mypath*"MCMC_samples_for_marker_effects.txt"
mapfile=mypath*"map42045.gwas.ihnv.csf2016.map"

"/Users/haocheng/Box Sync/5.Data/rainbowtrout/complete_genomic_data/map42045.gwas.ihnv.csf2016.map"

In [5]:
out=GWAS(file,header=false)
writedlm("modelfrequency.txt",out)

In [8]:
out=GWAS(file,mapfile,model,threshold=0.001,window_size="1 Mb",header=false)
writedlm("windowprobability.txt",out)

Compute posterior probability that window explains more than 0.001 of genetic variance


Progress: 100%|█████████████████████████████████████████| Time: 0:01:38


In [49]:
println("posterior mean for variance of 1:family_date")
println(mean(vec(output["MCMC samples for: variance of 1:family_date"])))

posterior mean for variance of 1:family_date
2.8730952852458245


In [36]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of marker effects"
  "MCMC samples for residual variance"
  "MCMC samples for: variance of 1:family_date"
  "MCMC samples for polygenic effects var-cov parameters"
  "Posterior mean of location parameters"

In [51]:
mean(output["MCMC samples for: variance of 1:family_date"])

2.8730952852458245